# Proof of concept: Calculate sun angle
Shadow length and time of day have been collected from http://centrodedescargas.cnig.es. Check
whether these can calculate a hub height that approximately matches the identified hub heights.

In [ ]:
import os

import dotenv
import numpy as np
import pandas as pd
from skyfield import api

dotenv.load_dotenv('../.env')
dotenv.load_dotenv('../.env.secret')

sites = pd.read_csv('../data/poc_measurements.csv')

In [ ]:
sites = sites.assign(
    date_string = lambda x: x.date + ' ' + x.hora + os.environ.get('utc_offset'),
    datetime_local = lambda x: pd.to_datetime(x.date_string, dayfirst=True),
    datetime_utc = lambda x: x.datetime_local.dt.tz_convert("UTC")
)
sites

In [ ]:
ephemeris = api.load('de421.bsp')
earth, sun = ephemeris['earth'], ephemeris['sun']
for i, site in sites.iloc[:4].iterrows():
    observer = earth + api.wgs84.latlon(latitude_degrees=site.latitude, longitude_degrees=site.longitude)
    time = api.load.timescale().from_datetime(site.datetime_utc)
    altitude, _, _  = observer.at(time).observe(sun).apparent().altaz()
    sites.loc[i, ["altitude_degrees", "altitude_radians"]]= altitude.degrees, altitude.radians
sites = sites.assign(
    estimated_hub_height=lambda x: np.tan(x.altitude_radians) * x.shadow_length
)
sites[["site", "latitude", "longitude", "hub_height", "shadow_length", "altitude_degrees",
       "estimated_hub_height"]].round(2)